In [2]:
import zipfile

local_zip = "rps.zip"
zip_ref = zipfile.ZipFile(local_zip, "r")
zip_ref.extractall("tmp/rps-train")
zip_ref.close()

local_zip = "rps-test-set.zip"
zip_ref = zipfile.ZipFile(local_zip, "r")
zip_ref.extractall("tmp/rps-test")
zip_ref.close()

In [5]:
import os

base_dir = "tmp/rps-train/rps"

rock_dir = os.path.join(base_dir, "rock")
paper_dir = os.path.join(base_dir, "paper")
scissors_dir = os.path.join(base_dir, "scissors")

In [8]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation="relu", input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation = "relu"),
    tf.keras.layers.Dense(3, activation="softmax")
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 17, 17, 128)       0

In [14]:
model.compile(loss ="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "tmp/rps-train/rps"
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

valid_dir = "tmp/rps-test/rps-test-set"
valid_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    class_mode="categorical",
    batch_size=128
)

valid_gen = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(150,150),
    class_mode="categorical",
    batch_size=128
)

Found 2520 images belonging to 3 classes.
Found 372 images belonging to 3 classes.


In [21]:
history = model.fit(train_gen,
                   epochs=25,
                   steps_per_epoch=20,
                   validation_data = valid_gen,
                   verbose=1,
                   validation_steps=3)

Epoch 1/25
20/20 [==============================] - 60s 3s/step - loss: 1.1285 - accuracy: 0.3397 - val_loss: 1.0639 - val_accuracy: 0.4597
Epoch 2/25
20/20 [==============================] - 48s 2s/step - loss: 1.3033 - accuracy: 0.4123
Epoch 3/25
20/20 [==============================] - 51s 3s/step - loss: 1.0057 - accuracy: 0.4766
Epoch 4/25
20/20 [==============================] - 63s 3s/step - loss: 0.9582 - accuracy: 0.5274
Epoch 5/25
20/20 [==============================] - 65s 3s/step - loss: 0.9134 - accuracy: 0.5694
Epoch 6/25
20/20 [==============================] - 66s 3s/step - loss: 0.7619 - accuracy: 0.6560
Epoch 7/25
20/20 [==============================] - 59s 3s/step - loss: 0.6785 - accuracy: 0.6750
Epoch 8/25
20/20 [==============================] - 60s 3s/step - loss: 0.5870 - accuracy: 0.7306
Epoch 9/25
20/20 [==============================] - 61s 3s/step - loss: 0.5114 - accuracy: 0.7742
Epoch 10/25
20/20 [==============================] - 62s 3s/step - loss: 0.4

In [40]:
zipfile.ZipFile("./rps-validation.zip", "r").extractall("./rps-validation")

In [45]:
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array

val_pics = os.listdir("./rps-validation")

for pic in val_pics:
    path = os.path.join("./rps-validation", pic)
    img = load_img(path, target_size=(150, 150))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    print(pic)
    print(classes)

1/1 [==============================] - 0s 21ms/step
paper-hires1.png
[[1. 0. 0.]]
1/1 [==============================] - 0s 21ms/step
paper-hires2.png
[[1. 0. 0.]]
1/1 [==============================] - 0s 22ms/step
paper1.png
[[1. 0. 0.]]
1/1 [==============================] - 0s 22ms/step
paper2.png
[[1. 0. 0.]]
1/1 [==============================] - 0s 23ms/step
paper3.png
[[1. 0. 0.]]
1/1 [==============================] - 0s 22ms/step
paper4.png
[[1. 0. 0.]]
1/1 [==============================] - 0s 21ms/step
paper5.png
[[1. 0. 0.]]
1/1 [==============================] - 0s 21ms/step
paper6.png
[[1. 0. 0.]]
1/1 [==============================] - 0s 22ms/step
paper7.png
[[1. 0. 0.]]
1/1 [==============================] - 0s 21ms/step
paper8.png
[[1. 0. 0.]]
1/1 [==============================] - 0s 22ms/step
paper9.png
[[1. 0. 0.]]
1/1 [==============================] - 0s 23ms/step
rock-hires1.png
[[0. 1. 0.]]
1/1 [==============================] - 0s 22ms/step
rock-hires2.png
[[0